In [6]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd
import warnings

warnings.filterwarnings("ignore") 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from cycles_signal_process import (
    calc_average_signal
)
from features_process import (
 
    get_waves_peak,
)
from pathlib import Path
import neurokit2 as nk
from tqdm import tqdm
import numpy as np

channel_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']


def get_age_group(age, age_groups):
    if age_groups is None:
        return age
    
    age_groups_sorted = sorted(age_groups)
    age_key = None
    
    for i in range(len(age_groups_sorted)-1):
        if age_groups_sorted[i] <= age < age_groups_sorted[i+1]:
            age_key = age_groups_sorted[i]
            break
    
    if age_key is None and age >= age_groups_sorted[-1]:
        age_key = age_groups_sorted[-1]
    
    return age_key


def get_average_signals_by_age(
    processed_dir, 
    fs=500,
    npz_files=None,
    output_dir='ptb_xl_average_signals',
    age_groups=None,
    method="dwt",
    output_dir_peaks='ptb_xl_peaks', 
    calc_waves_peak=False
):
    if npz_files is None:
        npz_files = list(processed_dir.glob("*.npz"))
        print(f"Found {len(npz_files)} patient files")
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    age_data = {}
    
    for file_path in tqdm(npz_files, desc="Processing patients"):
        output_dir_peaks = processed_dir.parent / output_dir_peaks / method
        try:
            data = np.load(file_path, allow_pickle=True)
            signal = data['signal']
            patient_id = int(data['patient_id'].item())
            age = int(data['age'].item()) if 'age' in data else None
            
            if age is None:
                continue

            age_key = get_age_group(age, age_groups)
            if age_key is None:
                continue
                
            if age_key not in age_data:
                age_data[age_key] = {channel: [] for channel in channel_names}
            
            waves_peak_filename = output_dir_peaks / f"{file_path.stem}_features.npz" #TODO _features -> _peaks
            if waves_peak_filename.exists() and calc_waves_peak==False:
                loaded_waves_peak = np.load(waves_peak_filename, allow_pickle=True)
                #waves_peak_info = {key: loaded_waves_peak[key] for key in loaded_waves_peak.files}
                waves_peak_info = {
                    key: loaded_waves_peak[key].item() for key in loaded_waves_peak if not key.startswith('__')
                }
            for i, channel_name in enumerate(channel_names):
                lead_signal = signal[i]
                signal_cleaned = nk.ecg_clean(lead_signal, sampling_rate=fs)
                
                waves_peak = get_waves_peak(signal_cleaned, fs, method=method)
                r_peaks = waves_peak['ECG_R_Peaks']
                
                if len(r_peaks) > 0:
                    avg_signal, _, _ = calc_average_signal(signal_cleaned, r_peaks, fs)
                    age_data[age_key][channel_name].append(avg_signal)
                    
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    return age_data

def save_age_group_data(age_data, output_dir):
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    np.savez(output_dir / "age_group_info.npz", age_data=age_data)
    
    for age, channels_data in age_data.items():
        age_dir = output_dir / str(age)
        age_dir.mkdir(exist_ok=True)
        
        for channel, signals in channels_data.items():
            if len(signals) > 0:
                final_avg_signal = np.mean(signals, axis=0)
                np.savez(
                    age_dir / f"{channel}_avg.npz",
                    signal=final_avg_signal,
                    age=age,
                    channel=channel,
                    num_patients=len(signals)
                )
    
    print(f"Results saved to {output_dir}")
    return output_dir

In [8]:
from paths import data_dir

In [9]:
data_dir, data_dir.exists()

(WindowsPath('C:/Users/lenovo/Desktop/sci/data/ecg'), True)

In [10]:
dataset = 'ptb_xl'
path_to_zip=data_dir / dataset
frequency = 500

channel_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
fs = 500

In [13]:
average_signals = get_average_signals_by_age(path_to_zip / 'ptb_xl_npz')

Found 18868 patient files


Processing patients:   1%|          | 105/18868 [00:44<2:10:43,  2.39it/s]

Error processing C:\Users\lenovo\Desktop\sci\data\ecg\ptb_xl\ptb_xl_npz\ecg_10108.npz: integer division or modulo by zero


Processing patients:   1%|          | 123/18868 [00:51<1:41:23,  3.08it/s]

Error processing C:\Users\lenovo\Desktop\sci\data\ecg\ptb_xl\ptb_xl_npz\ecg_10124.npz: cannot convert float NaN to integer


...

Processing patients: 100%|█████████▉| 18820/18868 [2:12:22<00:22,  2.17it/s]

Error processing C:\Users\lenovo\Desktop\sci\data\ecg\ptb_xl\ptb_xl_npz\ecg_9949.npz: cannot convert float NaN to integer


Processing patients: 100%|█████████▉| 18821/18868 [2:12:22<00:18,  2.51it/s]

Error processing C:\Users\lenovo\Desktop\sci\data\ecg\ptb_xl\ptb_xl_npz\ecg_995.npz: cannot convert float NaN to integer


Processing patients: 100%|██████████| 18868/18868 [2:12:49<00:00,  2.37it/s]


In [14]:
def save_age_group_data(age_data, output_dir):
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    np.savez(output_dir / "age_group_signal.npz", age_data=age_data)
    
    for age, channels_data in age_data.items():
        age_dir = output_dir / str(age)
        age_dir.mkdir(exist_ok=True)
        
        for channel, signals in channels_data.items():
            if len(signals) > 0:
                final_avg_signal = np.mean(signals, axis=0)
                np.savez(
                    age_dir / f"{channel}_avg.npz",
                    signal=final_avg_signal,
                    age=age,
                    channel=channel,
                    num_patients=len(signals)
                )
    
    print(f"Results saved to {output_dir}")
    return output_dir

save_age_group_data(average_signals, path_to_zip / 'signal_age_group')


Results saved to C:\Users\lenovo\Desktop\sci\data\ecg\ptb_xl\signal_age_group


WindowsPath('C:/Users/lenovo/Desktop/sci/data/ecg/ptb_xl/signal_age_group')

In [25]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

def plot_average_signals(age_data, fs=500, plot_dir='ptb_xl_plots'):
       
    for age_group, channels_data in age_data.items():
        for channel, signals in channels_data.items():
            if not signals:
                continue
                
            signals_array = np.array(signals)
            mean_signal = np.mean(signals_array, axis=0)
            median_signal = np.median(signals_array, axis=0)
            std_signal = np.std(signals_array, axis=0)
            time = np.arange(len(mean_signal)) / fs
            
            plt.figure(figsize=(12, 6))
            plt.plot(time, mean_signal, label='Mean Signal')
            plt.fill_between(time, mean_signal - std_signal, mean_signal + std_signal, alpha=0.3, label='Standard Deviation')
            plt.title(f"Age Group {age_group} - {channel}\nAverage ECG Signal")
            plt.xlabel("Time (s)")
            plt.ylabel("Amplitude")
            plt.legend()
            plt.grid(True)
            safe_age = str(age_group).replace(" ", "_").replace("-", "_")
            safe_channel = channel.replace(" ", "_").replace("-", "_")
            plt.savefig(plot_dir / f"avg_{safe_age}_{safe_channel}.png")
            plt.close()
            
            plt.figure(figsize=(12, 6))
            plt.plot(time, median_signal, label='Median Signal')
            plt.fill_between(time, median_signal - std_signal, median_signal + std_signal, alpha=0.3, label='Standard Deviation')
            plt.title(f"Age Group {age_group} - {channel}\nMedian ECG Signal")
            plt.xlabel("Time (s)")
            plt.ylabel("Amplitude")
            plt.legend()
            plt.grid(True)

            save_dir = Path(plot_dir) / f'{safe_age}'
            save_dir.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_dir / f"median_{safe_age}_{safe_channel}.png")
            plt.close()
        #break


plot_average_signals(average_signals, plot_dir = path_to_zip / 'plots' / 'signal_age_group' / '1_y')

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from pathlib import Path

def plot_all_average_signals(age_data, fs=500, plot_dir='ptb_xl_plots', channel='I'):
    plot_dir = Path(plot_dir)
    plot_dir.mkdir(parents=True, exist_ok=True)
    
    filtered_age_data = {
        age: channels 
        for age, channels in age_data.items() 
        if isinstance(age, (int, float)) and age <= 100
    }
    
    if not filtered_age_data:
        print("No age groups <= 100 years found")
        return
    
    sorted_age_groups = sorted(filtered_age_data.keys())
    min_age = min(sorted_age_groups)
    max_age = max(sorted_age_groups)
    norm = Normalize(vmin=min_age, vmax=max_age)
    cmap = plt.get_cmap('viridis')
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    lines = []
    for age_group in sorted_age_groups:
        if channel not in filtered_age_data[age_group]:
            continue
            
        signals = filtered_age_data[age_group][channel]
        if not signals:
            continue
            
        mean_signal = np.mean(np.array(signals), axis=0)
        time = np.arange(len(mean_signal)) / fs
        
        line, = ax.plot(time, mean_signal, 
                       color=cmap(norm(age_group)),
                       alpha=0.8,
                       linewidth=1.5,
                       label=f'{age_group} years')
        lines.append(line)
    
    if not lines:
        plt.close()
        return
    
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, ax=ax).set_label('Age (years)')
    
    ax.set_title(f'Average ECG Signals by Age Group (Channel {channel})')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.grid(True, alpha=0.3)
    
    safe_channel = channel.replace(" ", "_").replace("-", "_")
    plt.savefig(plot_dir / f'all_avg_signals_{safe_channel}.png', dpi=300, bbox_inches='tight')
    plt.close()

plot_all_average_signals(average_signals, plot_dir = path_to_zip / 'plots' / 'signal_age_group' / '1_y'/'all')

In [32]:
def plot_all_average_signals_for_channels(age_data, fs=500, plot_dir='ptb_xl_plots', channel_names=None):
    if channel_names is None:
        channel_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    
    plot_dir = Path(plot_dir)
    plot_dir.mkdir(parents=True, exist_ok=True)
    
    filtered_age_data = {
        age: channels 
        for age, channels in age_data.items() 
        if isinstance(age, (int, float)) and age <= 100
    }
    
    if not filtered_age_data:
        print("No age groups <= 100 years found")
        return
    
    sorted_age_groups = sorted(filtered_age_data.keys())
    min_age = min(sorted_age_groups)
    max_age = max(sorted_age_groups)
    norm = Normalize(vmin=min_age, vmax=max_age)
    cmap = plt.get_cmap('viridis')
    
    for channel in channel_names:
        fig, ax = plt.subplots(figsize=(14, 8))
        
        lines = []
        for age_group in sorted_age_groups:
            if channel not in filtered_age_data[age_group]:
                continue
                
            signals = filtered_age_data[age_group][channel]
            if not signals:
                continue
                
            mean_signal = np.mean(np.array(signals), axis=0)
            time = np.arange(len(mean_signal)) / fs
            
            line, = ax.plot(time, mean_signal, 
                          color=cmap(norm(age_group)),
                          alpha=0.8,
                          linewidth=1.5,
                          label=f'{age_group} years')
            lines.append(line)
        
        if not lines:
            plt.close()
            continue
        
        sm = ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        fig.colorbar(sm, ax=ax).set_label('Age (years)')
        
        ax.set_title(f'Average ECG Signals by Age Group (Channel {channel})')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Amplitude')
        ax.grid(True, alpha=0.3)
        
        safe_channel = channel.replace(" ", "_").replace("-", "_")
        plt.savefig(plot_dir / f'all_avg_signals_{safe_channel}.png', dpi=300, bbox_inches='tight')
        plt.close()

plot_all_average_signals_for_channels(
    average_signals, 
    plot_dir=path_to_zip / 'plots' / 'signal_age_group' / '1_y' / 'all',
    channel_names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
)

In [38]:
def plot_all_channels_for_age_groups(age_data, fs=500, plot_dir='ptb_xl_plots', channel_names=None, highlight_channels=['II']):
    if channel_names is None:
        channel_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    
    plot_dir = Path(plot_dir)
    plot_dir.mkdir(parents=True, exist_ok=True)
    
    filtered_age_data = {
        age: channels 
        for age, channels in age_data.items() 
        if isinstance(age, (int, float)) and age <= 100
    }
    
    #colors = plt.cm.tab20(np.linspace(0, 1, len(channel_names)))
    colors = plt.cm.gist_ncar(np.linspace(0, 1, len(channel_names)))
    colors[0] = (0, 0.75, 1, 1) 
    colors[-1] = (0, 0, 0, 1)
    
    for age_group, channels_data in filtered_age_data.items():
        if not any(channels_data.get(channel, []) for channel in channel_names):
            continue
            
        fig, ax = plt.subplots(figsize=(16, 8))
        all_signals = []
        for i, channel in enumerate(channel_names):
            signals = channels_data.get(channel, [])
            if not signals:
                continue
                
            mean_signal = np.mean(np.array(signals), axis=0)
            time = np.arange(len(mean_signal)) / fs
            linewidth = 3.5 if channel in highlight_channels else 1.5
            linestyle = '-' if channel in highlight_channels else '--'
            
            ax.plot(time, mean_signal, 
                   color=colors[i],
                   alpha=0.9,
                   linewidth=linewidth,
                   linestyle=linestyle,
                   label=channel)
            
            all_signals.append(mean_signal)
        
        if not all_signals:
            plt.close()
            continue
        
        ax.set_title(f'Average ECG Signals for Age {age_group} years')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Amplitude')
        ax.legend(loc='upper right')
        ax.grid(True, alpha=0.3)
        
        safe_age = str(age_group).replace(" ", "_").replace("-", "_")
        plt.savefig(plot_dir / f'all_channels_age_{safe_age}.png', dpi=300, bbox_inches='tight')
        plt.close()
plot_all_channels_for_age_groups(
    average_signals,
    plot_dir=path_to_zip / 'plots' / 'signal_age_group' / '1_y' / 'all_channels',
    channel_names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
)

In [ ]:

"""
def plot_ecg_cycle_with_features(cycle, signal, fs, cycle_num=0):
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # 1. Автоматическое определение границ цикла --------------------------------------
    available_points = []
    for point in ['ECG_P_Onsets', 'ECG_Q_Peaks', 'ECG_R_Peaks', 'ECG_S_Peaks', 'ECG_T_Offsets']:
        if point in cycle and len(cycle[point]) > 0:
            available_points.append(int(cycle[point][0]))
    
    if not available_points:
        print(f"Цикл {cycle_num}: нет точек для отрисовки")
        return fig
    
    # Границы цикла (с запасом 10% по краям)
    start = max(0, min(available_points) - int(0.1*fs))
    end = min(len(signal), max(available_points) + int(0.1*fs))
    
    # 2. Отрисовка сигнала ----------------------------------------------------------
    time = np.arange(start, end) / fs
    ax.plot(time, signal[start:end], 'b-', linewidth=1.5, label='ЭКГ сигнал')
    
    # 3. Разметка точек (только существующие) ---------------------------------------
    point_style = {
        'ECG_P_Onsets': ('<', 'green', 'P начало'),
        'ECG_P_Peaks': ('o', 'lime', 'P пик'),
        'ECG_P_Offsets': ('>', 'darkgreen', 'P конец'),
        'ECG_Q_Peaks': ('x', 'red', 'Q'),
        'ECG_R_Peaks': ('o', 'black', 'R'),
        'ECG_S_Peaks': ('x', 'blue', 'S'),
        'ECG_T_Onsets': ('<', 'purple', 'T начало'),
        'ECG_T_Peaks': ('o', 'magenta', 'T пик'),
        'ECG_T_Offsets': ('>', 'darkviolet', 'T конец')
    }
    
    for point, (marker, color, label) in point_style.items():
        if point in cycle and len(cycle[point]) > 0:
            x = cycle[point][0] / fs
            y = signal[int(cycle[point][0])]
            ax.plot(x, y, marker=marker, color=color, markersize=10, label=label)
    
    # 4. Отрисовка интервалов (только для существующих пар) -------------------------
    def draw_interval_if_exists(start_point, end_point, y_pos, label):
        if (start_point in cycle and len(cycle[start_point]) > 0 and 
            end_point in cycle and len(cycle[end_point]) > 0):
            x1 = cycle[start_point][0] / fs
            x2 = cycle[end_point][0] / fs
            ax.add_patch(Rectangle((x1, y_pos), x2-x1, 0.1, alpha=0.3, color='gray'))
            ax.text((x1+x2)/2, y_pos+0.15, f'{label}\n{(x2-x1)*1000:.1f} мс', 
                    ha='center', va='bottom', fontsize=9, bbox=dict(facecolor='white', alpha=0.7))
    
    y_min = np.min(signal[start:end])
    draw_interval_if_exists('ECG_P_Onsets', 'ECG_P_Offsets', y_min-0.3, 'P')
    draw_interval_if_exists('ECG_Q_Peaks', 'ECG_S_Peaks', y_min-0.5, 'QRS')
    draw_interval_if_exists('ECG_Q_Peaks', 'ECG_T_Offsets', y_min-0.7, 'QT')
    
    # 5. Настройка графика ----------------------------------------------------------
    ax.set_xlabel('Время (с)', fontsize=12)
    ax.set_ylabel('Амплитуда (мВ)', fontsize=12)
    ax.set_title(f'Цикл {cycle_num}', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Убираем дубликаты в легенде
    handles, labels = ax.get_legend_handles_labels()
    unique_labels = dict(zip(labels, handles))
    ax.legend(unique_labels.values(), unique_labels.keys(), loc='upper right')
    
    plt.tight_layout()
    return fig

def plot_averaged_signal(avg_signal, before_r, after_r, fs):
    if avg_signal is None:
        print("Невозможно построить график: недостаточно данных")
        return
    
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # 1. Временная ось (в мс относительно R-пика)
    time = (np.arange(len(avg_signal)) - before_r) / fs * 1000  # мс
    
    # 2. Отрисовка сигнала
    ax.plot(time, avg_signal, 'b-', linewidth=2, label='Усредненный ЭКГ')
    
    # 3. Разметка ключевых элементов
    ax.axvline(x=0, color='r', linestyle='--', label='R-пик (0 мс)')
    ax.axvline(x=-before_r, color='gray', linestyle=':', label=f'Начало окна (-{before_r/fs*1000:.0f} мс)')
    ax.axvline(x=after_r, color='gray', linestyle=':', label=f'Конец окна (+{after_r/fs*1000:.0f} мс)')
    
    # 4. Настройка графика
    ax.set_xlabel('Время относительно R-пика (мс)', fontsize=12)
    ax.set_ylabel('Амплитуда (мВ)', fontsize=12)
    ax.set_title(f'Усредненный PQRST комплекс (окно ±{before_r/fs*1000:.0f}/{after_r/fs*1000:.0f} мс)', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.5)
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

def plot_averaged_ecg(signal, cycles, fs):
    # 1. Подготовка данных для усреднения
    r_peaks = [int(cycle['ECG_R_Peaks'][0]) for cycle in cycles if 'ECG_R_Peaks' in cycle]
    if len(r_peaks) < 2:
        print("Недостаточно циклов для усреднения")
        return
    
    before_r = int(0.2 * fs)  # 200 мс до R
    after_r = int(0.5 * fs)   # 500 мс после R
    segments = []
    
    for r in r_peaks:
        start = max(0, r - before_r)
        end = min(len(signal), r + after_r)
        segment = signal[start:end]
        
        # Выравнивание по длине (если циклы у границ сигнала)
        if len(segment) < (before_r + after_r):
            pad_width = (before_r + after_r) - len(segment)
            segment = np.pad(segment, (0, pad_width), mode='constant')
        
        segments.append(segment)
    
    # 3. Усреднение и построение
    avg_signal = np.mean(segments, axis=0)
    time = (np.arange(len(avg_signal)) / fs) - 0.2  # Время относительно R-пика
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(time, avg_signal, 'b-', linewidth=2, label='Усредненный сигнал')
    
    # 4. Разметка характерных точек (медианные положения)
    point_labels = {
        'ECG_P_Peaks': ('P', 'green'),
        'ECG_Q_Peaks': ('Q', 'red'),
        'ECG_S_Peaks': ('S', 'blue'),
        'ECG_T_Peaks': ('T', 'purple')
    }
    
    for point_key, (label, color) in point_labels.items():
        points = []
        for cycle in cycles:
            if point_key in cycle and len(cycle[point_key]) > 0:
                points.append(cycle[point_key][0])
        
        if len(points) > 0:
            median_pos = np.median(points) - r_peaks[0]  # Относительно первого R-пика
            median_time = median_pos / fs
            y_val = np.interp(median_pos, np.arange(len(avg_signal)), avg_signal)
            ax.plot(median_time, y_val, 'o', color=color, markersize=8, label=label)
    
    # 5. Настройка графика
    ax.axvline(x=0, color='black', linestyle='--', label='R-пик')
    ax.set_xlabel('Время относительно R-пика (с)')
    ax.set_ylabel('Амплитуда (мВ)')
    ax.set_title(f'Усредненный PQRST комплекс (n={len(cycles)} циклов)')
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend()
    
    plt.tight_layout()
    plt.show()
"""

'\ndef plot_ecg_cycle_with_features(cycle, signal, fs, cycle_num=0):\n    fig, ax = plt.subplots(figsize=(15, 8))\n\n    # 1. Автоматическое определение границ цикла --------------------------------------\n    available_points = []\n    for point in [\'ECG_P_Onsets\', \'ECG_Q_Peaks\', \'ECG_R_Peaks\', \'ECG_S_Peaks\', \'ECG_T_Offsets\']:\n        if point in cycle and len(cycle[point]) > 0:\n            available_points.append(int(cycle[point][0]))\n\n    if not available_points:\n        print(f"Цикл {cycle_num}: нет точек для отрисовки")\n        return fig\n\n    # Границы цикла (с запасом 10% по краям)\n    start = max(0, min(available_points) - int(0.1*fs))\n    end = min(len(signal), max(available_points) + int(0.1*fs))\n\n    # 2. Отрисовка сигнала ----------------------------------------------------------\n    time = np.arange(start, end) / fs\n    ax.plot(time, signal[start:end], \'b-\', linewidth=1.5, label=\'ЭКГ сигнал\')\n\n    # 3. Разметка точек (только существующие) ----